In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import pandas as pd
import seaborn as sns
DATA_PATH = os.path.join("../input")

def load_data(fileName, data_path=DATA_PATH):
    csv_path = os.path.join(data_path, fileName)
    return pd.read_csv(csv_path)

In [ ]:
train_set = load_data("train.csv")
test_set = load_data("test.csv")
expected_results = load_data("gender_submission.csv")

In [ ]:
test_passenger_ids = {'PassengerId':  expected_results["PassengerId"], 'Survived': []}

In [ ]:
train_set.head()

In [ ]:
import seaborn as sns

sns.barplot(x="Pclass", y="Survived", data=train_set)
#Clearly, the passengers in class 1 and 2 were more likely to survive

In [ ]:
train_set["Pclass"] = train_set["Pclass"].fillna('0')
test_set["Pclass"] = test_set["Pclass"].fillna('0')

In [ ]:
sns.barplot(x="Sex", y="Survived", data=train_set)
# Sex of the surviving passengers is also important, as seen in the graph below

In [ ]:
mean_value = train_set['Age'].mean()
train_set["Age"] = train_set["Age"].fillna(mean_value)

In [ ]:
mean_value_test = test_set['Age'].mean()
test_set["Age"] = test_set["Age"].fillna(mean_value_test)

In [ ]:
train_set["Embarked"] = train_set["Embarked"].fillna('X')
test_set["Embarked"] = test_set["Embarked"].fillna('X')

In [ ]:
bins = (0, 10, 20, 30, 40, 50, 60, 100)
group_names = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-100']
train_set['Age'] = pd.cut(train_set['Age'], bins = bins, labels = group_names)
test_set['Age'] = pd.cut(test_set['Age'], bins = bins, labels = group_names)

In [ ]:
sns.barplot(x="SibSp", y="Survived", data=train_set)

In [ ]:
sns.barplot(x="Parch", y="Survived", data=train_set)

In [ ]:
sns.barplot(x="Embarked", y="Survived", data=train_set)
#From the graph below, we can also see that the passengers that embarked in Cherbourg were more likely to survive 

In [ ]:
train_set["Fare"] = train_set["Fare"].fillna(train_set["Fare"].mean())
test_set["Fare"] = test_set["Fare"].fillna(test_set["Fare"].mean())

In [ ]:
labels = train_set["Survived"]
train_set = train_set.drop(["Survived"], axis=1)

In [ ]:
#"Parch", "SibSp"
train_set["Parch"].fillna(train_set["Parch"].mean())
test_set["Parch"].fillna(test_set["Parch"].mean())

train_set["SibSp"].fillna(train_set["SibSp"].mean())
test_set["SibSp"].fillna(test_set["SibSp"].mean())

In [ ]:
# Definition of the CategoricalEncoder class, copied from PR #9151.
# Just run this cell, or copy it to your code, do not try to understand it (yet).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse
import numpy as np

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [ ]:
#Transform non-numerical values
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_attribs = ["Fare", "Parch", "SibSp"]
cat_attibs = ["Sex", "Pclass", "Age"]

scaler = StandardScaler()
scaler.fit(train_set['Fare'].values.reshape(-1, 1))
train_set['Fare'] = scaler.transform(train_set['Fare'].values.reshape(-1, 1))

scaler.fit(test_set['Fare'].values.reshape(-1, 1))
test_set['Fare'] = scaler.transform(test_set['Fare'].values.reshape(-1, 1))


num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs))
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attibs)),
    ('cat_encoder', CategoricalEncoder(encoding="onehot-dense"))
])

In [ ]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline)
])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_set, labels, test_size=0.2, random_state=42)
prepared_train_data = full_pipeline.fit_transform(X_train)
prepared_val_data = full_pipeline.fit_transform(X_val)

prepared_test_data = full_pipeline.fit_transform(test_set)

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import RandomizedSearchCV

import numpy as np

models = [
    {
        'model': BernoulliNB(), 
        'model_name': 'BernoulliNB', 
        'params': 
        {
            'binarize': [binarize for binarize in np.arange(0, 1, 0.01)],
            'fit_prior': [True, False]
        },
        'best_model': None,
        'accuracy': 0.0
    },
    {
        'model': DecisionTreeClassifier(), 
        'model_name': 'DecisionTreeClassifier', 
        'params': 
        {
            'splitter': ['best', 'random'],
            'max_depth': [depth for depth in range(1, 300)],
            'min_samples_split': [min_samples for min_samples in np.arange(0.01, 1, 0.01)],
            'min_samples_leaf': [min_leaf for min_leaf in range(1, 10)]
        },
        'best_model': None,
        'accuracy': 0.0
    },
    {
        'model': RandomForestClassifier(), 
        'model_name': 'RandomForestClassifier', 
        'params': 
        {
            'n_estimators': [n for n in range(1, 300)],
            'max_depth': [depth for depth in range(1, 300)],
            'min_samples_split': [min_samples for min_samples in np.arange(0.01, 1, 0.01)],
            'min_samples_leaf': [min_leaf for min_leaf in range(1, 100)]
        },
        'best_model': None,
        'accuracy': 0.0
    },
    {
        'model': KNeighborsClassifier(), 
        'model_name': 'KNeighborsClassifier', 
        'params': 
        {
            'n_neighbors': [n for n in range(1, 200)],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'leaf_size': [size for size in range(1, 200)]
        },
        'best_model': None,
        'accuracy': 0.0
    },
    {
        'model': SVC(probability=True), 
        'model_name': 'SVC', 
        'params': 
        {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [c for c in np.arange(0.1, 1, 0.1)],

        },
        'best_model': None,
        'accuracy': 0.0
    },
    {
        'model': SGDClassifier(),
        'model_name': 'SGD_clf',
        'params':
        {
            'loss': ['hinge', 'modified_huber', 'log', 'squared_hinge', 'squared_loss'],
            'warm_start': [True, False]
        },
        'best_model': None,
        'accuracy': 0.0
    }
]

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def model_selection(X, y, model_info):
    """
    Test various estimators.
    """    

    rf_random = RandomizedSearchCV(estimator=model_info['model'], param_distributions=model_info['params'])    
    rf_random.fit(X, y)
    model_info['best_model'] = rf_random.best_estimator_
    
    expected  = y
    predicted = model_info['best_model'].predict(prepared_val_data)
    
    model_info['accuracy'] = accuracy_score(y_val, predicted)
    
    print('MSE:', mean_squared_error(y_val, predicted))
    print('Model:', model_info['model_name'], ',', 'Precision:', (accuracy_score(y_val, predicted)) * 100, '%')
    print(cross_validate(model_info['model'], model_info['model_name'], X, y))

def cross_validate(estimator, estimator_name, X, y):
    scores = cross_val_score(estimator, X, y, scoring='neg_mean_squared_error', cv=10)
    plot_scores(scores, estimator_name)
    
def plot_scores(scores, estimator_name):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
    
def get_estimators(accuracy=0.8):
    best_estimators = [
        (estimator['best_model'].__class__.__name__, estimator['best_model']) 
        for estimator in models 
        if estimator['accuracy'] > accuracy
    ]
    print(best_estimators)
    return best_estimators

In [ ]:
for model in models:
    model_selection(prepared_train_data, y_train, model)

In [ ]:
best_estimators = get_estimators(0.8)

In [ ]:
from sklearn.ensemble import VotingClassifier

vote_clf = VotingClassifier(
    estimators=best_estimators,
    voting='soft'
)

In [ ]:
vote_clf.fit(prepared_train_data, y_train)

In [ ]:
predictions = vote_clf.predict(prepared_val_data)

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_val, predictions)
rmse = np.sqrt(mse)
rmse

In [ ]:
final_predictions = vote_clf.predict(prepared_test_data)

In [ ]:
test_passenger_ids["Survived"] = final_predictions

In [ ]:
df = pd.DataFrame(data = test_passenger_ids)
df

In [ ]:
df.to_csv("results.csv")